# Predicting sale price of flat

## Background
In Singapore, close to 80% of the population live in public housing. These are heavily subsidised by the government and as such, strict rules have been implemented to ensure the subsidies are not being misused. One such rule is that every couple is not to own more than a single subsidised property at any one time. 

## Project Motivation
I am interested in finding the potential selling price of my flat in 3 years. The main objective of this project is to provide an estimate of the expected retail price for the flat in due time. In this project, I will use a variety of regression models to make predictions on the potential selling price, rather than to take the prediction of a single model. This is to showcase the relative effectiveness of the various models for the present purpose -- predict sale price of a flat given historical transaction data.

## Data Collection
Transaction data of public flats have been dilligently chronicled [here](https://data.gov.sg/dataset/resale-flat-prices). In the final version of the project, the data will be obtained from a json object from an API call. In the initial version(s), the entire dataset will be downloaded and fed to the project in a `pandas` dataframe. Operations such as cleaning and grouping will then be performed on the dataframe itself.

### Inspiration from GovTech DSAID blog

Before going on to the project I would like to acknowledge that a huge part of this project is inspired from a [blog](https://medium.com/dsaid-govtech/wanted-data-scientists-how-we-designed-our-first-case-interview-9fd4eaa4607c) written by the [Data Science and Artificial Intelligence Division (DSAID)](https://medium.com/dsaid-govtech) of a governmental agency called GovTech. They have been pushed to the forefront since the COVID-19 pandemic hit Singapore. A familiar (and valuable) product developed by DSAID was the TraceTogether Token (and its associated App). 

For those who are unfamiliar, the token collects data via bluetooth signals, such as proximity data, duration in proximity data and places visited (We are supposed to scan for entry at the entrance of facilities like shopping malls). In the early days, this token expidited contact tracing of people exposed to others who were subsequently identified as COVID-positive, facilitating rapid isolation to supress the spread of the virus. If an individual had been shown to be in the proximity of a (subsequently identified) positive individual using data from the token, he/she will also receive a message to perform health monitoring for the requisite number of days.

After seeing the impact they have on the life on Singaporeans, after launching useful products like the TraceTogether Token and App, I thought what they are doing is very meaningful and creates impact in our everyday lives. I am sure the job there will be sufficiently motivating in and of itself.

I would want to start off with using the OneMap API to geocode the lat-long coordinates just for the fun of it. Intuitively, I do not think that should affect the selling price anymore than the 'Town' feature. Including it is purely for practice on calling information from APIs.

First, the essential libraries...

In [ ]:
import pandas as pd
import numpy as np
import requests as req
from matplotlib import pyplot as plt
import json
import seaborn as sns
import re
from datetime import date
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

Based on the information on [HDB API](https://data.gov.sg/dataset/resale-flat-prices), we can find the API query string as follows. We will perform our entire analysis using our entire dataset of 92,270 entries(correct as of the time of writing).


In [ ]:

query_str = 'https://data.gov.sg/api/action/datastore_search?resource_id=f1765b54-a209-4718-8d38-a39237f502b3&limit=500'
response = req.get(query_str)
response.status_code

A side note on the response obtained from the **requests** library. A *status_code* of 200 implies that the request was successful. Otherwise the response code would have been *404 NOT FOUND*. Other possible status codes can be found [here](https://en.wikipedia.org/wiki/List_of_HTTP_status_codes).

In [ ]:
data = json.loads(response.content)
type(data)

In [ ]:
len(data['result']['records'])

## The Data
As mentioned, in the initial version(s), the data is obtained from a downloaded `.csv` file and read into a dataframe. This will be changed to an API call in the final version. After that, check that the dataset has been properly read and explore some features of the data such as the number of columns, the number of rows and the type of every column.

Even though data from the 1990's are available, they are probably outdated due to the numerous policy implementations over the years. We will only use the most recent data from 2017 onwards.

In [ ]:
df = pd.read_csv("data/resale-flat-prices-based-on-registration-date-from-jan-2017-onwards.csv")
df.head()

### Brief explanation of columns
The various dates within the dataset may appear strage to someone not staying in Singapore. Most HDB flats come with a default 99-year lease from the government. At the end of the 99 years, the government reserve the right to repossess the land on which the flat is built for other developments. This is largely due to the lack of geographic mass of the country, which drives up the price of land and properties.

An additional note is that the `month` column contains data on when the associated flat is sold on the second-hand market. Feel free to refer to the metadata file for explanations to the context of other columns.

In [ ]:
df.shape

We see that there are a total of 131560 entries, which should be enough for a small project.

### Data type of columns

In [ ]:
df.info()

As seen above, most of the columns are classified as `object`, even though some of them contains useful numerical data, such as `flat_type`, `storey_range` and `month`. With all else being equal, the higher the value of `storey_range`, the higher the resale price. This is due to Singaporeans' inherent preference for flats on a higher level. In this section, we will transform them into the appropriate forms.

First we extract the `year` and `month` value from the resale date, and thereafter dropping it since our model does not take in `datetime` type as a feature.

Columns such as `town`, `flat_type` and `flat_model` will be One-hot encoded as they contain information about different categories of flats, with no inherent order. Features like `storey_range` on the other land will be transformed using `OrdinalEncoder` since the value of the category has an effect on the resale price (take it from a Singaporean who has stayed here all his life!).

In [ ]:
# df.rename({'month': 'sale_date'}, axis=1, inplace=True)
df['sale_date'] = pd.to_datetime(df['month'], format="%Y-%m")
df['sale_year'] = df['sale_date'].dt.year
df['sale_month'] = df['sale_date'].dt.month
df.drop(['sale_date', 'month'], axis=1, inplace=True)

### Handling `remaining_lease`
The amount of remaining lease affects the resale price in an indirect way. As mentioned, the maximum lease for HDB flats is currently at 99 years. Hence, the longer the remaining lease, the more valuable the flat is.

In [ ]:
df['years_remaining'] = df['remaining_lease'].apply(lambda x: int(x.split()[0]))
df['months_remaining'] = df['remaining_lease'].apply(lambda x: int(x.split(" ")[-2]))
df.drop(["remaining_lease", "lease_commence_date"], axis=1, inplace=True)
df.head()

In [ ]:
df['storey_range'].unique()

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

storey_cat = ['01 TO 03', '04 TO 06', '07 TO 09', '10 TO 12', '13 TO 15', '16 TO 18', 
                '19 TO 21', '22 TO 24', '25 TO 27', '28 TO 30', '31 TO 33', '34 TO 36',
                '37 TO 39', '40 TO 42', '43 TO 45', '46 TO 48', '49 TO 51']
oe = OrdinalEncoder(categories=[storey_cat], dtype=int)
df["storey"] = oe.fit_transform(df[["storey_range"]])

In [ ]:
df.columns

In [ ]:
ohe = OneHotEncoder(dtype=int)
ohe_features = ['town', 'flat_type', 'flat_model']
flat_type_encoding = ohe.fit_transform(df[ohe_features]).toarray()
flat_type_encoding_df = pd.DataFrame(flat_type_encoding, columns=ohe.get_feature_names())
df = df.join(flat_type_encoding_df)
df.head()

Next we work on encoding the other columns, such as `town` and `flat_model`, once again using `OneHotEncoder`. However, `storey_range` will be encoded with `OrdinalEncoder` instead due to its ordinal nature. In general, the higher the level, the better the resale price.

### Column distribution

In [ ]:
df.describe().transpose()

In [ ]:
df.isna().sum()

Under the `df.info()`, we see that most of our dataframe columns contain objects. We need to handle them one by one. Thanks to the data collection and meticulous efforts, there are no missing values in the columns, and this means we do not have to worry about filling in the *NaN*'s with mean or median values.

In [ ]:
print(df['town'].unique())
df['town'].nunique()

We see that our data set contains around 26 unique town locations. Given the property mantra of "location, location, location", the idea will be to tag them by the first 3 digits of their postal code to turn the data in this column from `object` to `int` so as to be fed into our model.

In [ ]:
plt.figure(figsize=(10, 5))
sns.distplot(df['resale_price'])
plt.xlabel('Resale Price (S$M)')

Apart from finding out how the prices are distributed, we would also want to find out how the other features correlate with the resale price. This is so that we can sort the features in terms of the correlation it has with the resale price subsequently.

In [ ]:
corrMat = df.corr()
sns.heatmap(corrMat, annot=True)

There are not many cells to plot here since the correlation can only be calculated for numerical features. Here we see that amongst the numerical features, **SIZE DOES MATTER**! 

Of course the correlation of PSM should be largely the same as floor area since it is derived as the ratio of the resale price and floor area.

In [ ]:
plt.figure(figsize=(15, 8))
ax = sns.boxplot(x=df['town'], y=df['resale_price']/1E6)
ax.set_xticklabels(ax.get_xticklabels(),rotation=45)
ax.set_ylabel('Resale Price (S$M)')

Now this plot looks messy since it is arranged in alphabetical order of the town the flat resides in. Let's try to order this by the median price instead and see if the plot looks tidier.

In [ ]:
resale_median = df.groupby(['town']).median().reset_index()
resale_median.sort_values(by='resale_price', ascending=True, inplace=True, ignore_index=True)
resale_median['resale_price'] /= 1000000 

In [ ]:
sns.boxplot(x='town', y='resale_price', data=resale_median)

In [ ]:
df.drop(["town", "flat_type", "flat_model", "storey_range", "block", "street_name"], axis=1, inplace=True)
df.columns

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.drop('resale_price', axis=1), df['resale_price'], test_size=0.25)

In [ ]:
lr = LinearRegression()
lr.fit(X_train, y_train)
lr.score(X_test, y_test)

In [ ]:
mean_squared_error(y_test, lr.predict(X_test))

# Concluding Thoughts
The mean_squared_error of $59.3B translates to a RMS error of around $24.3K. Given that the resale prices across all flat types ranges around $300 - 600K, this translates to an error of almost 10%. The error can be further reduced by properly setting the hyperparameters of the `LinearRegression` model, or processing the features with `PolynomialFeatures` before fitting. 

In the interim, let's recall the initial motivation for this project -- to find the projected price of my flat given the relevant features. I shall do that below, but for a 4-room flat at the "01 TO 03" storey range in Yishun, does anyone want to make a guess?

In [ ]:
myflat = pd.read_csv("data/myflat.csv", header=0)
myflat.head()

In [ ]:
myflat['sale_year'] = myflat['month'].apply(lambda x: int(x.split("-")[0]))
myflat['sale_month'] = myflat['month'].apply(lambda x: int(x.split("-")[-1]))
myflat.drop('month', axis=1, inplace=True)
myflat['storey'] = oe.transform(myflat[["storey_range"]])

In [ ]:
myflat_encoding = ohe.transform(myflat[ohe_features]).toarray()
myflat_encoding_df = pd.DataFrame(myflat_encoding, columns=ohe.get_feature_names())
myflat = myflat.join(myflat_encoding_df)
myflat['years_remaining'] = myflat['remaining_lease'].apply(lambda x: int(x.split()[0]))
myflat['months_remaining'] = myflat['remaining_lease'].apply(lambda x: int(x.split(" ")[-2]))
myflat.head()

In [ ]:
myflat.drop(["remaining_lease", "lease_commence_date", "town", "flat_type", "flat_model", "storey_range", "block", "street_name"], axis=1, inplace=True)
myflat.columns

In [ ]:
assert len(df.columns) == len(myflat.columns) + 1
myflat.head()

In [ ]:
myflat = myflat[X_train.columns]    # switch the order of columns in myflat to match that of X_train
print(f"\nPredicted price for my flat is ${lr.predict(myflat)[0]:.2f}")

Not bad! The model predicted a sale price of around $483K, which seems to match the records of recent transactions around the area (as of July 2022). I will make a mental note to update the actual transacted price once I sell my flat, so do remember to star this project to be kept in the loop. And remember, this comes even without including additional information such as proximity to transport hubs, famous schools and venues of entertainment such as shopping hubs. I would say that it is pretty impressive! 

What do you think?